In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
import xgboost as xgb

# pull feature engineering notebook
# add your own feature engineering functions to features.py and import them
# only rule is the function must take a dataframe and return a dataframe (with your new features)
from features import *
from utils import *

color = sns.color_palette()
%matplotlib inline

In [ ]:
df_train = pd.read_json("train.json")
df_test = pd.read_json("test.json")

### PRE-PROCESSING

In [ ]:
# from features.py
scrub_and_engineer = [
                    scrub,
                    basic_numeric_features,
                    n_log_price,
                    n_expensive,
                    count_caps,
                    scrub_features,
                    dist_to_nearest_tube,
                    dist_to_nearest_college,
                    add_neighbor_features_72,
                      ]

for func in scrub_and_engineer:
    try:
        df_train = func(df_train)
        df_test = func(df_test)
    except Exception as e:
        print e
        continue

In [ ]:
exclude = ['price']
feats_to_train = [x for x in df_train.columns.tolist() if df_train[x].dtype not in ['O', '<M8[ns]'] and x not in exclude]
feats_to_train

### Hyperparameters

In [ ]:
# Cross val
test_size=0.20

# Random Forest
n_estimators=1000

### Baseline training

In [ ]:
base_features = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]
X = df_train[base_features]
y = df_train["interest_level"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size)

clf = RandomForestClassifier(n_estimators=n_estimators)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
baseline_logloss = log_loss(y_val, y_val_pred)

In [ ]:
baseline_logloss

### Training the new model

In [ ]:
# filter out any object/string + timestamp variables and train the random forest on numerical columns
# feats_to_train = [x for x in df_train.columns.tolist() if df_train[x].dtype not in ['O', '<M8[ns]']] 
X = df_train[feats_to_train]
y = df_train["interest_level"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size)

test_clf = RandomForestClassifier(
    n_estimators=n_estimators,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1)
test_clf.fit(X_train, y_train)
y_val_pred = test_clf.predict_proba(X_val)
test_logloss = log_loss(y_val, y_val_pred)

In [ ]:
test_logloss

### Testing for raw improvement

In [ ]:
if baseline_logloss - test_logloss > 0:
    print "Model improved, save and submit"
else:
    print "Use baseline model, did not improve"

In [ ]:
# test RF accuracy
accuracy_score(test_clf.predict(X_val), y_val)

In [ ]:
# test RF feature importance
plot_feature_importance(test_clf, feats_to_train)

In [ ]:
# baseline RF accuracy
accuracy_score(clf.predict(X_val), y_val)

In [ ]:
# baseline RF feature importance
plot_feature_importance(clf, base_features)

### Clean up and output a submission file

In [ ]:
sub = output(df_test, clf, feats_to_train)

In [ ]:
sub.to_csv("rf-Tom.csv", index=False)

In [ ]:
# submission = pd.read_csv('submission_rf.csv')
# len(submission)